In [1]:
import sqlite3
import pandas as pd
from pandas import DataFrame, Series

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

sqladb = pd.read_csv('adult.csv',names=["age", "workclass", "fnlwgt", "education","education-num","marital-status","occupation",
                                        "relationship","race","sex","capital-gain","capital-loss","hours-per-week"
                                       ,"native-country","label"])
#print(sqladb)
connection = sqlite3.connect('sqladb')

connection.execute("DROP TABLE IF EXISTS adult")

cursor = connection.execute("""CREATE TABLE IF NOT EXISTS adult (
            id int,
            age   int,
            workclass   varchar(40),
            fnlwgt   int,
            education   varchar(40),
            education_num   int,
            marital_status   varchar(40),
            occupation   varchar(20),
            relationship   varchar(40),
            race   varchar(20),
            sex   varchar(10),
            capital_gain   int,
            capital_loss   int,
            hours_per_week   int,
            native_country   varchar(50),
            label    varchar(10))""")
#question 1


 

In [2]:
insert_query ='INSERT INTO adult VALUES("%s","%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s")'
for index, row in sqladb.iterrows():
    
    connection.execute(insert_query % (index,row['age'], row['workclass'], 
                                       row['fnlwgt'], row['education'],row['education-num'],row['marital-status'],row['occupation'],row['relationship']
                                      ,row['race'],row['sex'],row['capital-gain'],row['capital-loss'],row['hours-per-week'],row['native-country'],row['label']))

# Commit is important to reflect the changes
connection.commit()


#### 1. Select 10 records from the adult sqladb

In [3]:
cursor = connection.execute("SELECT * FROM adult LIMIT 10")
#print(cursor.fetchall())
for row in cursor:
    print("======")
    print(row)


(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K')
(1, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')
(2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(3, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(4, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K')
(5, 37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
(6, 49, ' Private', 160187, ' 9th', 5, ' Married-spouse-absent', 

#### 2. Show me the average hours per week of all men who are working in private sector

In [4]:
cursor.execute("SELECT AVG(hours_per_week) FROM adult WHERE workclass = ' Private' ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)


(40.267095523440254,)


#### 3. Show me the frequency table for education, occupation and relationship, separately

#### education

In [5]:
cursor.execute("SELECT education , COUNT(education) frequency FROM adult GROUP BY education ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(' 10th', 933)
(' 11th', 1175)
(' 12th', 433)
(' 1st-4th', 168)
(' 5th-6th', 333)
(' 7th-8th', 646)
(' 9th', 514)
(' Assoc-acdm', 1067)
(' Assoc-voc', 1382)
(' Bachelors', 5355)
(' Doctorate', 413)
(' HS-grad', 10501)
(' Masters', 1723)
(' Preschool', 51)
(' Prof-school', 576)
(' Some-college', 7291)


#### occupation

In [6]:
cursor.execute("SELECT occupation, COUNT(occupation) frequency FROM adult GROUP BY occupation ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(' ?', 1843)
(' Adm-clerical', 3770)
(' Armed-Forces', 9)
(' Craft-repair', 4099)
(' Exec-managerial', 4066)
(' Farming-fishing', 994)
(' Handlers-cleaners', 1370)
(' Machine-op-inspct', 2002)
(' Other-service', 3295)
(' Priv-house-serv', 149)
(' Prof-specialty', 4140)
(' Protective-serv', 649)
(' Sales', 3650)
(' Tech-support', 928)
(' Transport-moving', 1597)


#### relationship

In [7]:
cursor.execute("SELECT relationship, COUNT(relationship) frequency FROM adult GROUP BY relationship ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(' Husband', 13193)
(' Not-in-family', 8305)
(' Other-relative', 981)
(' Own-child', 5068)
(' Unmarried', 3446)
(' Wife', 1568)


#### 4. Are there any people who are married, working in private sector and having a masters degree

In [8]:
cursor.execute(" SELECT COUNT(marital_status) count FROM adult WHERE marital_status IN (' Married-civ-spouse', ' Married-spouse-absent' , ' Married-AF-spouse')")
myresult = cursor.fetchall()
for x in myresult:
    if x[0] > 0:
        print('Yes')
    else:
        print('No')


Yes


#### 5. What is the average, minimum and maximum age group for people working in different sectors

In [9]:
cursor.execute(" SELECT workclass as sector, AVG(age) averageAge, MIN(age) minAge, MAX(age) maxAge  FROM adult GROUP BY workclass ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(' ?', 40.960239651416124, 17, 90)
(' Federal-gov', 42.590625, 17, 90)
(' Local-gov', 41.75107501194458, 17, 90)
(' Never-worked', 20.571428571428573, 17, 30)
(' Private', 36.797585477617204, 17, 90)
(' Self-emp-inc', 46.01702508960574, 17, 84)
(' Self-emp-not-inc', 44.96969696969697, 17, 90)
(' State-gov', 39.43605546995377, 17, 81)
(' Without-pay', 47.785714285714285, 19, 72)


#### 6. Calculate age distribution by country

In [10]:
cursor.execute(" SELECT native_country,age FROM adult GROUP BY native_country,age ")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(' ?', 17)
(' ?', 18)
(' ?', 19)
(' ?', 20)
(' ?', 21)
(' ?', 22)
(' ?', 23)
(' ?', 24)
(' ?', 25)
(' ?', 26)
(' ?', 27)
(' ?', 28)
(' ?', 29)
(' ?', 30)
(' ?', 31)
(' ?', 32)
(' ?', 33)
(' ?', 34)
(' ?', 35)
(' ?', 36)
(' ?', 37)
(' ?', 38)
(' ?', 39)
(' ?', 40)
(' ?', 41)
(' ?', 42)
(' ?', 43)
(' ?', 44)
(' ?', 45)
(' ?', 46)
(' ?', 47)
(' ?', 48)
(' ?', 49)
(' ?', 50)
(' ?', 51)
(' ?', 52)
(' ?', 53)
(' ?', 54)
(' ?', 55)
(' ?', 56)
(' ?', 57)
(' ?', 58)
(' ?', 59)
(' ?', 60)
(' ?', 61)
(' ?', 62)
(' ?', 63)
(' ?', 64)
(' ?', 65)
(' ?', 66)
(' ?', 67)
(' ?', 68)
(' ?', 69)
(' ?', 70)
(' ?', 72)
(' ?', 74)
(' ?', 76)
(' ?', 81)
(' ?', 90)
(' Cambodia', 18)
(' Cambodia', 25)
(' Cambodia', 27)
(' Cambodia', 28)
(' Cambodia', 32)
(' Cambodia', 34)
(' Cambodia', 35)
(' Cambodia', 36)
(' Cambodia', 37)
(' Cambodia', 40)
(' Cambodia', 42)
(' Cambodia', 46)
(' Cambodia', 48)
(' Cambodia', 50)
(' Cambodia', 51)
(' Cambodia', 65)
(' Canada', 17)
(' Canada', 18)
(' Canada', 19)
(' Canada', 20)

(' Outlying-US(Guam-USVI-etc)', 47)
(' Outlying-US(Guam-USVI-etc)', 59)
(' Outlying-US(Guam-USVI-etc)', 63)
(' Peru', 17)
(' Peru', 18)
(' Peru', 20)
(' Peru', 21)
(' Peru', 22)
(' Peru', 23)
(' Peru', 24)
(' Peru', 25)
(' Peru', 26)
(' Peru', 27)
(' Peru', 29)
(' Peru', 30)
(' Peru', 33)
(' Peru', 34)
(' Peru', 35)
(' Peru', 37)
(' Peru', 41)
(' Peru', 44)
(' Peru', 46)
(' Peru', 48)
(' Peru', 49)
(' Peru', 64)
(' Peru', 69)
(' Philippines', 17)
(' Philippines', 18)
(' Philippines', 19)
(' Philippines', 20)
(' Philippines', 21)
(' Philippines', 22)
(' Philippines', 23)
(' Philippines', 24)
(' Philippines', 25)
(' Philippines', 26)
(' Philippines', 27)
(' Philippines', 28)
(' Philippines', 29)
(' Philippines', 30)
(' Philippines', 31)
(' Philippines', 32)
(' Philippines', 33)
(' Philippines', 34)
(' Philippines', 35)
(' Philippines', 36)
(' Philippines', 37)
(' Philippines', 38)
(' Philippines', 39)
(' Philippines', 40)
(' Philippines', 41)
(' Philippines', 42)
(' Philippines', 43)
(' 

#### 7. Compute a new column as ‘Net-Capital-Gain’ from the two columns ‘capital-gain’ and ‘capital-loss’

In [34]:
cursor.execute("SELECT capital_gain-capital_loss Net_Capital_Gain from adult")
myresult = cursor.fetchall()
for x in myresult:
    print( "Net_Capital_Gain is {}".format(x))

Net_Capital_Gain is (2174,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (14084,)
Net_Capital_Gain is (5178,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-2042,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1408,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Cap

Net_Capital_Gain is (0,)
Net_Capital_Gain is (7298,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (3411,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1977,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (10605,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1887,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (15024,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (7298,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Ga

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (7298,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (99999,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain i

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (1506,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (3325,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (3137,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1902,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1902,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capi

...

Net_Capital_Gain is (99999,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (99999,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1977,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1590,)
Net_Capital_Gain is (3781,)
Net_Capital_Gain is (3325,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (8614,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (27828,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-2377,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (14084,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_G

Net_Capital_Gain is (-1590,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1590,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (15024,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (15024,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1848,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1740,)
Net_Capital_Gain is (-1887,)
Net_Capital_Gain is (0

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (4064,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (6418,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (5455,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-2174,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (7298,)
Net_Capit

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (15024,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (5178,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (10520,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (99999,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capita

Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-2258,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-2444,)
Net_Capital_Gain is (15024,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1408,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (1055,)
Net_Capital_Gain is (7298,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (-1602,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (10520,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (0,)
Net_Capital_Gain is (4101,)
Net_Capital_Gain is (0,)
Net_Capital_Gain 

#### Task 2 
##### Create an sqlalchemy engine using a sample from the data set

In [12]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy as sqla

engine = create_engine('sqlite:///sqladb.db', echo=False)
Base = declarative_base()

sql = sqla.text("DROP TABLE IF EXISTS adult_user ")
engine.execute(sql)


class Adults(Base):
    __tablename__ = 'adult_user'
    id = Column(Integer, primary_key=True)
    age = Column(Integer)
    workclass = Column(String)
    fnlwgt = Column(String)
    education = Column(String)
    education_num = Column(Integer)
    marital_status = Column(String)
    occupation = Column(String)
    relationship = Column(String)
    race = Column(String)
    sex = Column(String)
    capital_gain = Column(Integer)
    capital_loss = Column(Integer)
    hours_per_week = Column(Integer)
    native_country = Column(String)
    labbel = Column(String)        

In [13]:
Base.metadata.create_all(engine)

In [14]:
adults = Adults(id=1,age=19, workclass=' Private', 
                fnlwgt=12,education='Bachelors',
                education_num=13,
               marital_status='Married-civ-spouse',
               occupation='Exec-managerial',
               relationship='Husband',
               race='White',
               sex='Male',
               capital_gain='0',
               capital_loss='0',
               hours_per_week='13',
               native_country='United-States',
               labbel='<=50K'
               )
print(adults.age)

adults1 = Adults(id=2,age=29, workclass=' Self-emp-not-inc', 
                fnlwgt=83311,education='Bachelors',
                education_num=13,
               marital_status='Married-civ-spouse',
               occupation='Exec-managerial',
               relationship='Husband',
               race='White',
               sex='Male',
               capital_gain='0',
               capital_loss='0',
               hours_per_week='13',
               native_country='United-States',
               labbel='<=50K'
               )
print(adults1.age)

adults2 = Adults(id=3,age=39, workclass=' Self-emp-not-inc', 
                fnlwgt=83311,education='Bachelors',
                education_num=13,
               marital_status='Married-civ-spouse',
               occupation='Exec-managerial',
               relationship='Husband',
               race='White',
               sex='Male',
               capital_gain='0',
               capital_loss='0',
               hours_per_week='13',
               native_country='United-States',
               labbel='<=50K'
               )
print(adults2.age)

adults3 = Adults(id=4,age=49, workclass=' Self-emp-not-inc', 
                fnlwgt=83311,education='Bachelors',
                education_num=13,
               marital_status='Married-civ-spouse',
               occupation='Exec-managerial',
               relationship='Husband',
               race='White',
               sex='Male',
               capital_gain='0',
               capital_loss='0',
               hours_per_week='13',
               native_country='United-States',
               labbel='<=50K'
               )
print(adults3.age)

19
29
39
49


#### Write two basic update queries

In [15]:

from sqlalchemy.orm import sessionmaker


In [16]:
session_maker = sessionmaker(bind=engine)

In [17]:
session_maker = sessionmaker()

In [18]:
session_maker.configure(bind=engine) 

In [19]:
session = session_maker()

In [20]:
session.add_all([adults,adults1,adults2,adults3])


In [21]:
#session = session_maker()

for instance in session.query(Adults):
     print(instance.id,instance.age, instance.workclass)

1 19  Private
2 29  Self-emp-not-inc
3 39  Self-emp-not-inc
4 49  Self-emp-not-inc


In [22]:
#session = session_maker()
adult_user = session.query(Adults).all()
adult_user

#### Update

In [23]:
from sqlalchemy.sql.expression import update
#session = session_maker()
conn = engine.connect()
stmt = update(Adults).where(adults.age==19).values(education='Masters')
print(stmt)
session.execute(stmt)
session.commit()
print("adults---",adults.age,adults.education)

#session = session_maker()
conn = engine.connect()
stmt = update(Adults).where(adults1.age==29).values(education='Masters')
print(stmt)
session.execute(stmt)
session.commit()
print("adults1---",adults1.age,adults1.education)

UPDATE adult_user SET education=:education WHERE true
adults--- 19 Masters
UPDATE adult_user SET education=:education WHERE true
adults1--- 29 Masters


#### Write two delete queries

In [24]:
adult_user_1=session.query(Adults).filter_by(age = 19).first()
adult_user_1.age

adult_user_2=session.query(Adults).filter_by(age = 29).first()
adult_user_2.age

session.delete(adult_user_1)
session.delete(adult_user_2)
session.commit()


In [25]:
for instance in session.query(Adults):
     print(instance.id,instance.age, instance.workclass)

3 39  Self-emp-not-inc
4 49  Self-emp-not-inc


#### Write two filter queries

In [26]:
adult_user_1=session.query(Adults).filter_by(id = 3).first()
print("adult_user_1 Id ",adult_user_1.id)

adult_user_2=session.query(Adults).filter_by(age = 49).first()
print("adult_user_2 Age ",adult_user_2.age)

adult_user_1 Id  3
adult_user_2 Age  49


#### Write two function queries

In [27]:
from sqlalchemy import func
from sqlalchemy.sql import label

results = session.query(Adults.occupation,
label('members', func.count(Adults.occupation)),label('total_hours_per_week',
func.sum(Adults.hours_per_week))).group_by(Adults.occupation).all()

for result in results:
    print ("{} has {} members with a total_hours_per_week of {}".
    format(result.occupation, result.members, result.total_hours_per_week))


Exec-managerial has 2 members with a total_hours_per_week of 26
